In [ ]:
import pdfplumber
import pandas as pd
from pdfreader import SimplePDFViewer
from bs4 import BeautifulSoup
import requests
from geopy.distance import geodesic
import os
from dotenv import load_dotenv



budget_adopted_2023_2024_url = 'https://www.seattleschools.org/wp-content/uploads/2023/09/Budget-Book-Final_Web-Version-Adopted.pdf'
budget_summary_2023_2024_url = 'https://www.seattleschools.org/wp-content/uploads/2023/09/FY24-Adopted-Budget-Summary-Tables.pdf'
budget='budget_adopted_2023_2024.pdf'
budget_summary='budget_summary_2023_2024.pdf'

load_dotenv()

########################################################################################
#   Functions listed here: 
########################################################################################

def get_lat_long(address, api_key):
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and len(data['results']) > 0:
            location = data['results'][0]['geometry']['location']
            return location['lat'], location['lng']
    return None, None


def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).miles


def extract_tables_from_pdf(pdf_path, page_num):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_num]
        tables = page.extract_tables()
        
        dataframes = []
        for table in tables:
            df = pd.DataFrame(table[1:], columns=table[0])
            dataframes.append(df)
        return dataframes

def find_dataframe_with_column(dfs, column_name):
    for idx, df in enumerate(dfs):
        if column_name in df.columns:
            return idx
    return -1  # Return -1 if the column name is not found in any DataFrame

def extract_text_from_pdf(pdf_path, page_num):
    with open(pdf_path, "rb") as file:
        viewer = SimplePDFViewer(file)
        viewer.navigate(page_num + 1)
        viewer.render()
        return "".join(viewer.canvas.strings)

def get_budget_table(page_num, include_school=True, pivoted=True):
    # First extract the tables from the PDF
    dataframes = extract_tables_from_pdf(budget, page_num)

    # Find the DataFrame that contains the budget table, it varies by page because of nuances with PDFPlumber
    budget_table_index = find_dataframe_with_column(dataframes,'School Year\nFunding Type 21-22 22-23 23-24')
    budget_table_b = dataframes[budget_table_index][[None]][None].str.split(' ', expand=True)
    budget_table_a = dataframes[budget_table_index].iloc[:, 0:1]

    # Overwrite the column names with the correct ones, Merge the two DataFrames    
    budget_table_a.columns = ['Budget']
    budget_table_b.columns = ['2021-22', '2022-23', '2023-24']
    budget_table = pd.merge(budget_table_a, budget_table_b, left_index=True, right_index=True)
    
    
    # Append the string BUDGET to indicate it is the budget data
    budget_table['Budget'] = budget_table['Budget'].apply(lambda x: x + ' (BUDGET)')

    # Clean up the DataFrame and cast numeric values to floats.
    budget_table['2021-22'] = budget_table['2021-22'].str.replace('$', '').str.replace(',', '').astype(float)
    budget_table['2022-23'] = budget_table['2022-23'].str.replace('$', '').str.replace(',', '').astype(float)
    budget_table['2023-24'] = budget_table['2023-24'].str.replace('$', '').str.replace(',', '').astype(float)


    if pivoted: 
        budget_table = budget_table.pivot_table(columns='Budget', values=['2023-24','2022-23','2021-22'], aggfunc='sum') 
        budget_table.columns.name = 'Index'
        budget_table = budget_table.reset_index().rename(columns={'index':'Year'})


    # Find the school name and add it to the DataFrame using a weird string in the PDF to identify the school name
    raw_text = extract_text_from_pdf(budget,page_num)
    if include_school:
        budget_table['School'] = raw_text[0:raw_text.find('A.2023-24')]
        col_to_move = budget_table.columns[-1]
        budget_table = budget_table[[col_to_move] + [col for col in budget_table.columns if col != col_to_move]]

    # Reorder the columns, Return the final dataframe
    return budget_table


def get_demographics_table(page_num, include_school=True, pivoted=True):
    # First extract the tables from the PDF
    dataframes = extract_tables_from_pdf(budget, page_num)

    # Find the DataFrame that contains the budget table, it varies by page because of nuances with PDFPlumber
    demographics_table_index = find_dataframe_with_column(dataframes,'School Year\n21-22 22-23 23-24')
    demographics_table_b = dataframes[demographics_table_index][[None]][None].str.split(' ', expand=True)
    demographics_table_a = dataframes[demographics_table_index].iloc[:, 0:1]    

    # Overwrite the column names with the correct ones, Merge the two DataFrames    
    demographics_table_a.columns = ['Demographic']
    demographics_table_b.columns = ['2021-22', '2022-23', '2023-24']
    demographics_table = pd.merge(demographics_table_a, demographics_table_b, left_index=True, right_index=True)

    # Append the string ENROLLMENT to indicate it is the enrollment data
    demographics_table['Demographic'] = demographics_table['Demographic'].apply(lambda x: x + ' (ENROLLMENT)')

    # Find the school name and add it to the DataFrame using a weird string in the PDF to identify the school name
    raw_text = extract_text_from_pdf(budget,page_num)

    if pivoted: 
        demographics_table = demographics_table.pivot_table(columns='Demographic', values=['2023-24','2022-23','2021-22'], aggfunc='sum') 
        demographics_table.columns.name = 'Index'
        demographics_table = demographics_table.reset_index().rename(columns={'index':'Year'})

    if include_school:
        demographics_table['School'] = raw_text[0:raw_text.find('A.2023-24')]
        col_to_move = demographics_table.columns[-1]
        demographics_table = demographics_table[[col_to_move] + [col for col in demographics_table.columns if col != col_to_move]]

    # Reorder the columns, Return the final dataframe
    return demographics_table


In [ ]:
########################################################################################
#   Build the Budget Table: 
########################################################################################
x = None

for i in range(67,129): #Pages 67-129 are for elementary schools
    if (x is None):
        x = get_budget_table(i)
    else:
        x = pd.concat([x,get_budget_table(i)], ignore_index=True)
    print(i)

In [ ]:
########################################################################################
#   Build the Demographics Table: 
########################################################################################
y = None

for i in range(67,129): #Pages 67-129 are for elementary schools
    if (y is None):
        y = get_demographics_table(i)
    else:
        y = pd.concat([y,get_demographics_table(i)], ignore_index=True)
    print(i)

In [ ]:
########################################################################################
#   Get Addresses and Lat/Longs 
########################################################################################
coords = None

for i in range(1,8): 

    # URL of the page to scrape    
    url = f'https://www.seattleschools.org/schools/type/elementary/page/{i}/'

    # Send a request to fetch the webpage content
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if the request was unsuccessful

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all elements that contain school information
    school_elements = soup.find_all('div', class_='list-item')

    # Prepare lists to store school names and addresses
    schools = []
    addresses = []

    # Loop through each school element to extract names and addresses
    for school_element in school_elements:
        name_element = school_element.find('h4', class_='list-item-title').find('a')
        address_element = school_element.find('address')

        if name_element and address_element:
            school_name = name_element.get_text(strip=True)
            address = address_element.get_text(separator=" ", strip=True)
            
            schools.append(school_name)
            addresses.append(address)

    # Create a DataFrame from the extracted data
    data = {
        'School': schools,
        'Full Address': addresses
    }
    if coords is None:
        coords = pd.DataFrame(data)
    else: 
        coords = pd.concat([coords,pd.DataFrame(data)], ignore_index=True) 

# Remove trailing text from the addresses
api_key = os.getenv('GOOGLE_MAPS_API_KEY')
coords['Full Address'] = coords['Full Address'].apply(lambda x: x[0:x.find('About')])
coords['Latitude'], coords['Longitude'] = zip(*coords['Full Address'].apply(lambda x: get_lat_long(x, api_key)))

In [ ]:
########################################################################################
#   Calculate nearest schools: 
########################################################################################


for index, row in coords.iterrows():
    min_distance = float('inf')
    closest_school = None
    school_coords = (row['Latitude'], row['Longitude'])

    for idx, other_row in coords.iterrows():
        if index != idx:
            other_school_coords = (other_row['Latitude'], other_row['Longitude'])
            distance = calculate_distance(school_coords, other_school_coords)
            if distance < min_distance:
                min_distance = distance
                closest_school = other_row['School']

    coords.at[index, 'Closest School'] = closest_school
    coords.at[index, 'Distance to Closest School (miles)'] = min_distance

In [ ]:
########################################################################################
#   Merge all DFs together, create one as 'z': 
########################################################################################


z = pd.merge(y,x, on=['School','Year'])
z = pd.merge(z,coords, on='School')

# Calculate helpful additional columns from existing data and cast numeric values to floats
z['Budget Efficiency'] = z['(BDG) Total Budget'].astype(float) / z['(ENR) Total AAFTE* Enrollment'].astype(float)
z['(ENR) Total AAFTE* Enrollment'] = z['(ENR) Total AAFTE* Enrollment'].astype(float)

z.to_csv('sps_data_extract.csv')



In [ ]:
########################################################################################
#   Sandbox and Playground Below: 
########################################################################################

In [ ]:
z[z['School'] == 'Decatur Elementary']

# Group by school and calculate the average budget efficiency
school_performance = z.groupby('School').agg({
    'Budget Efficiency': 'mean',
    '(ENR) Total AAFTE* Enrollment': 'mean',
    '(BDG) Bilingual Education': 'mean',
    '(BDG) Special Education': 'mean',
    '(BDG) General Education': 'mean'
}).reset_index()

# Sort schools by budget efficiency in descending order to identify top-performing schools
top_schools = school_performance.sort_values(by='Budget Efficiency', ascending=False).head(10)
top_schools

In [ ]:
# Calculate the change in enrollment for each school
enrollment_changes = z.pivot_table(index='School', columns='Year', values='(ENR) Total AAFTE* Enrollment')

# Calculate the difference between the first and last year for each school
enrollment_changes['Change'] = enrollment_changes.iloc[:, -1] - enrollment_changes.iloc[:, 0]

# Sort schools by enrollment change in ascending order to find the sharpest decline
sharpest_decline = enrollment_changes.sort_values(by='Change').head(10)

# Sort schools by enrollment change in descending order to find the highest increase
highest_increase = enrollment_changes.sort_values(by='Change', ascending=False).head(10)

sharpest_decline, highest_increase

In [ ]:

# Study of Cascadia Elementary School - which was closed in 2023
cascadia_data = z[z['School'] == 'Cascadia Elementrary']

# Extract relevant columns for Cascadia Elementary
cascadia_summary = cascadia_data[['Year', '(ENR) Total AAFTE* Enrollment', 'Budget Efficiency',
                                  '(BDG) Bilingual Education', '(BDG) Special Education', '(BDG) General Education']]

# Display Cascadia Elementary summary
cascadia_summary

# Detailed trends for Cascadia Elementary

# Calculate year-on-year changes in enrollment and budget efficiency for Cascadia
cascadia_data['Enrollment Change'] = cascadia_data['(ENR) Total AAFTE* Enrollment'].diff()
cascadia_data['Budget Efficiency Change'] = cascadia_data['Budget Efficiency'].diff()

# Calculate year-on-year changes in program funding
cascadia_data['Bilingual Education Change'] = cascadia_data['(BDG) Bilingual Education'].diff()
cascadia_data['Special Education Change'] = cascadia_data['(BDG) Special Education'].diff()
cascadia_data['General Education Change'] = cascadia_data['(BDG) General Education'].diff()

# Display detailed trends
cascadia_data[['Year', '(ENR) Total AAFTE* Enrollment', 'Enrollment Change', 'Budget Efficiency', 'Budget Efficiency Change',
               '(BDG) Bilingual Education', 'Bilingual Education Change', '(BDG) Special Education', 'Special Education Change',
               '(BDG) General Education', 'General Education Change']]

In [ ]:
cascadia_data[['Year', 'Enrollment Change', 'Budget Efficiency Change',
                'Bilingual Education Change', 'Special Education Change',
               'General Education Change']]

In [ ]:
# Examples to get Budget Tables: 
#get_budget_table(121,True) # Raw Budget Table
#get_budget_table(121,False).pivot_table(columns='Budget', values=['2023-24','2022-23','2021-22'], aggfunc='sum') # Pivoted Budget Table

#get_demographics_table(121,False) # Raw Demographics Table
#get_demographics_table(121,False).pivot_table(columns='Demographic', values=['2023-24','2022-23','2021-22'], aggfunc='sum') # Pivoted Demographics Table

# Example usage to extract tables from a PDF
#pdf_path = budget
#page_num = 69  # Page 62 in the document
#dataframes = extract_tables_from_pdf(pdf_path, page_num)

# Example usage to gather all text in a PDF page
#extract_text_from_pdf(budget, 121)
